In [1]:
%load_ext autoreload
%autoreload 2

In [41]:
from library import Plugin, CacheFiles
from library import get_help
from importlib import import_module
from pathlib import Path
import constants
import logging
from IPython.display import Image 


In [3]:
logging.root.setLevel('WARNING')

In [46]:
p = my_plugin.image

In [58]:
img_path.exists()

False

In [61]:
plugin_readme

PosixPath('plugins/word_clock/README.md')

In [62]:
plugins = get_help.get_modules()
cache = CacheFiles()
active_plugins = []
refresh_rate = 1
doc_path = Path('../documentation/')
img_path = Path(doc_path/'images')
plugin_path = Path('./plugins')

if not img_path.exists():
    img_path.mkdir(parents=True, exist_ok=True)

resolution = (800, 600)
for plugin in plugins:
    config = None
    module = None
    plugin_sample_img = None
    try:
        sample = import_module(f'{constants.plugins}.{plugin}.sample')
        module = import_module(f'{constants.plugins}.{plugin}')
    except ModuleNotFoundError:
        logging.warning(f'module "{plugin}" is missing a sample configuration; cannot process')
        continue
        
    config = sample.config        
    if config and module:
        layout = getattr(module.layout, config['layout'])
        my_plugin = Plugin(resolution=resolution, 
                           cache=cache, layout=layout,
                           update_function=module.update_function, config=config)
        my_plugin.refresh_rate = refresh_rate
        active_plugins.append(my_plugin)
        if 'kwargs' in config:
            my_plugin.update(**config['kwargs'])
        else:
            my_plugin.update()
#         display(my_plugin.image)
    
    # write out readme and save image
    plugin_readme = Path(doc_path/f'README_{plugin}.md')
    if my_plugin.image:
        plugin_sample_img = Path(img_path/f'{plugin}_sample.png')
        my_plugin.image.save(plugin_sample_img)
    with open(plugin_readme, 'w') as file:
        file.write(f'# mod_name\n')
        file.write(f'![sample image for plugin {plugin}]({plugin_sample_img})\n')
        file.write('```\n' + get_help.get_module_docs(module) + '\n```')
#         print(f"## {mod_name} ")
#         print('```\n' + get_help.get_module_docs(module) + '\n```')

    

SyntaxError: invalid syntax (<ipython-input-62-381514651180>, line 46)